In [262]:
import pandas as pd
import numpy as np
from clustering_alg import custom_clustering
import gensim.models
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

In [263]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_train.head()

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry


In [264]:
df_train.shape

(8695, 3)

In [265]:
# for comment in df_train.Comment:
#     df_train.loc[df_train['Comment']==comment,'Comment'] = comment.replace("\n"," ")
#     df_train.loc[df_train['Comment']==comment,'Comment'] = comment.replace("\\n"," ")
# df_train.iloc[0,1]

In [266]:
df_train

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry
...,...,...,...
8690,0x1e02,I make similar observations over the last week...,Biology
8691,0xc8d,You would know.,Biology
8692,0x723,Also use the correct number of sig figs,Chemistry
8693,0x667,"What about the ethical delimmas, groundbreaki...",Biology


In [267]:
df_train['Comment'].tolist()[0]

'A few things. You might have negative- frequency dependent selection going on where the least common phenotype, reflected by genotype, is going to have an advantage in the environment. For instance, if a prey animal such as a vole were to have a light and a dark phenotype, a predator might recognize the more common phenotype as food.  So if the light voles are more common, foxes may be keeping a closer eye out for light phenotypic voles, recognising them as good prey. This would reduce the light causing alleles due to increased predation and the dark genotypes would increase their proportion of the population until this scenario is reversed. This cycle continues perpetually. \\n\\nHowever, this is unlikely to be strictly yearly as it usually takes more time than a year for an entire populations allele frequencies to change enough to make a large enough difference to alter fitness. \\n\\nMore likely on a *year to year* basis, the population is experiencing fluctuating selection where a

In [268]:
comments_list = df_train['Comment'].tolist()
w2v_data = list()
for comment in comments_list:
    for sentence in sent_tokenize(comment):
        temp = []
        for word in word_tokenize(sentence):
            temp.append(word.lower())
        w2v_data.append(temp)    
w2v_data[0]

['a', 'few', 'things', '.']

In [269]:
df_train

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry
...,...,...,...
8690,0x1e02,I make similar observations over the last week...,Biology
8691,0xc8d,You would know.,Biology
8692,0x723,Also use the correct number of sig figs,Chemistry
8693,0x667,"What about the ethical delimmas, groundbreaki...",Biology


In [270]:
model = gensim.models.Word2Vec(w2v_data, min_count = 1, vector_size = 100, window = 2, hs=1)

In [271]:
print(model.wv.similarity('cesium','observations'),model.wv.similarity('ride','car'))

-0.08860872 0.6494742


In [276]:
words_vecs = pd.DataFrame(model.wv.vectors)
# clusters_count = 3
words_labels = custom_clustering(words_vecs)

MemoryError: 